---
# Chapter 10
## Sequence Hacking, Hashing, and Slicing

---

## Vector Take #1: Vector2d Compatible

---
### Example 10-1: Tests of Vector.\_\_init\_\_ and Vector.\_\_repr\_\_

In [ ]:
from vector_v1 import Vector
Vector([3.1, 4.2])

In [ ]:
Vector((3, 4, 5))

---
### Example 10-2: vector_v1.py: derived from vector2d_v1.py

See the file `vector_v1.py` for more details

---
### Example 10-3: Code from Example 1-1. reproduced here for convenience

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrechDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hears'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self) -> int:
        return len(self._cards)

    def __getitem__(self, position: int) -> Card:
        return self._cards[position]
        

In [ ]:
class SampleVector(Vector):

    def __getitem__(self, position):
        return self._components[position]

    def __len__(self):
        return len(self._components)


In [ ]:
v1 = SampleVector([3, 4, 5])
len(v1)

In [ ]:
v1[0], v1[-1]

In [ ]:
v7 = SampleVector(range(7))
v7[1:4]

## Vector Take #2: A Sliceable Sequence

### How Slicing Works

---
### Example 10-4: Checking out the behavior of \_\_getitem\_\_ and slices

In [ ]:
class MySeq:
    def __getitem__(self, position: int) -> object:
        return position

In [ ]:
s = MySeq()
s[1]

In [ ]:
s[1:4]

In [ ]:
s[1:4:2]

In [ ]:
s[1:4:2, 9]

In [ ]:
s[1:4:2, 7:9]

---
### Example 10-5: Inspecting the attributes of te slice class

In [ ]:
repr(slice)

In [ ]:
dir(slice)

In [ ]:
help(slice.indices)

In [ ]:
slice(None, 10, 2).indices(5)

In [ ]:
slice(-3, None, None).indices(5)

In [ ]:
print('ABCDE'[:10:2])
print('ABCDE'[:5:2])

print('ABCDE'[-3:])
print('ABCDE'[2:5:1])


### A Slice-Aware \_\_getitem\_\_

---
### Example 10-6: Part of vector_v2.py: *\_\_len\_\_* and *\_\_getitem\_\_* methods added to Vector class from *vector_v1.py* (Example 10-2)

```python
def __len__(self):
    return len(self._components)

def __getitem__(self, index):
    cls = type(self)
    if isinstance(index, slice):
        return cls(self._components[index])
    elif isinstance(index, numbers.Integral):
        return self._components[index]
    else:
        msg = '{cls.__name__} indices mus be integers'
        return msg.format(cls=cls)
```

---
### Example 10-7: Tests of enhaced Vector.gettiem from Example 10-6

In [ ]:
from vector_v2 import Vector

v7 = Vector(range(7))
v7[-1]

In [ ]:
v7[1:4]

In [ ]:
v7[-1:]

In [ ]:
v7[1, 2]

## Vector Take #3: Dynamic Attribute Access

---
### Example 10-8: Part of `vector_v3.py`: *\_\_getattr\_\_* method added to Vector class from *\_\_vector_v2.py\_\_*

```python
 shorcut_names = 'xyzt'

 def __getattr__(self, name: str) -> object:
     cls = type(self)
     if len(name) == 1:
         pos = cls.shortcut_names.find(name)
         if 0 <= pos <= len(self._components):
             return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
```

---
### Example 10-9: Inappropriate behavior: assigning to v x raises no error, but introduces an inconsistency

In [1]:
from vector_v3 import Vector

v = Vector([0.0, 1.0, 2.0, 3.0, 4.0])
v.x

0.0

In [2]:
v.x = 10
v.x

10

In [3]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

---
### Example 10-10: Part of vector v3.py: *\_\_setattr\_\_* method in Vector class

```python
def __setattr__(self, name: str, value: object) -> None:
    cls = type(self)

    if len(namr) == 1:
        if name in cls.shortcut_names:
            error = 'Read only attribute {attr_name!r}'
        elif name.islower():
            error = "Can-t set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''

        if error:
            msg = error.format(cls_name= cls.__name__, attr_name=name)
            raise AttributeError(msg, value)

```

In [4]:
from vector_v3_1 import Vector

v = Vector([0.0, 1.0, 2.0, 3.0, 4.0])
v.x

0.0

In [5]:
v.x = 10

AttributeError: ("Read only attribute 'x'", 10)

## Vector Take #4: Hashing and Faster ==

### Example 10-11: Three ways of calculating the accumulated xor of integers from 0 to 5

In [6]:
# Method 1:
n = 0
for i in range(1, 6):
    n ^= i

n

1

In [8]:
# Method 2:
import functools

n = functools.reduce(lambda a,b: a^b, range(1, 6))
n

1

In [10]:
# Method 3:
import functools
import operator

n = functools.reduce(operator.xor, range(1,6))
n

1

---
### Example 10-12:

In [12]:
from vector_v4 import Vector

v1 = Vector([1, 2])
hash(v1)

3

In [13]:
v4 = Vector([1, 2, 3, 4, 5])
hash(v4)

1

## Vector Take #5: Formatting

### Example